<a href="https://colab.research.google.com/github/MugulTum/Sentiment_analysis/blob/main/IMDB_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### BERT Model

Bidirectional Encoder Representations from Transformers (BERT) is a technique for NLP (Natural Language Processing) pre-training developed by Google. BERT was created and published in 2018 by Jacob Devlin and his colleagues from Google. Google is leveraging BERT to better understand user searches.

BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial taskspecific architecture modifications.

In [9]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf


In [10]:
data_train = pd.read_excel('/content/train.xlsx')
data_test = pd.read_excel('/content/test.xlsx')

In [11]:
data_train.shape, data_test.shape

((25000, 2), (25000, 2))

In [12]:
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [13]:
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [14]:
data_train.tail()

,Reviews,Sentiment
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg
24999,"So why does this show suck? Unfortunately, tha...",neg


In [15]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=data_train,
                                                     text_column='Reviews',
                                                     label_columns='Sentiment',
                                                     val_df = data_test,
                                                     maxlen=500,
                                                     preprocess_mode='bert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [16]:
model = text.text_classifier(name='bert',
                             train_data = (X_train, y_train),
                             preproc = preproc
                             )

Is Multi-Label? False
maxlen is 500


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


done.


In [17]:
learner = ktrain.get_learner(model=model, train_data = (X_train, y_train),
                   val_data = (X_test, y_test),
                   batch_size = 6
                   )

In [18]:
#finding the best learning rate
#learner.lr_find()
#learner.lr_plot()

In [19]:
learner.fit_onecycle(lr=2e-5, epochs=1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 4600s 1s/step - loss: 0.2524 - accuracy: 0.8967 - val_loss: 0.1044 - val_accuracy: 0.9663


In [20]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [22]:
data = ['this movie was horrible, the plot was really boring. the acting was great',
        'the film sucked, there was no plot and the acting was equally bad',
        'what a beautiful movie, the plot was awesome. the acting was really great']

In [23]:
predictor.predict(data)

['neg', 'neg', 'pos']

In [25]:
predictor.predict(data, return_proba=True)

array([[0.9898391 , 0.01016091],
       [0.9927596 , 0.00724044],
       [0.00319901, 0.996801  ]], dtype=float32)

In [26]:
predictor.get_classes()

['neg', 'pos']

In [27]:
predictor.save('/content/bert')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:
!zip -r /content/bert.zip /content/bert

  adding: content/bert/ (stored 0%)
  adding: content/bert/tf_model.preproc (deflated 48%)
  adding: content/bert/tf_model.h5 (deflated 11%)


In [30]:
predictor_load = ktrain.load_predictor('/content/bert')

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [31]:
predictor_load.get_classes()

['neg', 'pos']

In [32]:
predictor_load.predict(data)

['neg', 'neg', 'pos']